Analysing fidelities for various sweeps for N=3 Photons

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import sqrtm
import pickle
import glob
import os

In [9]:
# Function to load pickle file
def load_pkl(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

# Function to compute the complex phase of the off-diagonal element 10
def compute_phase(real_part, imag_part):
    return np.arctan2(imag_part, real_part)

# Function to calculate fidelity
def calculate_fidelity(rho, sigma):
    """
    Compute the fidelity between two density matrices.

    Parameters:
    rho (numpy.ndarray): The first density matrix (must be a square, Hermitian, positive semi-definite matrix).
    sigma (numpy.ndarray): The second density matrix (must be a square, Hermitian, positive semi-definite matrix).

    Returns:
    float: The fidelity between the two density matrices.
    """
    # Compute the square root of rho
    sqrt_rho = sqrtm(rho)
    
    # Compute the product sqrt(rho) * sigma * sqrt(rho)
    product = np.dot(sqrt_rho, np.dot(sigma, sqrt_rho))
    
    # Compute the square root of the product
    sqrt_product = sqrtm(product)
    
    # Take the trace and square the result
    trace_value = np.trace(sqrt_product)
    
    # Return the fidelity
    return np.real(trace_value)

In [3]:
# Load the dictionaries from the .pkl files
data = load_pkl("photon_correlations/n_3/n3_results_diag_nstart2_vstlength1_b0p07_twophotdet0_0.pkl")
data_off_diag = load_pkl("photon_correlations/n_3/n3_results_off_diag_nstart2_vstlength1_b0p07_twophotdet0_0.pkl")

# Print the keys of both dictionaries
print("Keys in data:", data.keys())
print("Keys in data_off_diag:", data_off_diag.keys())

# Merge dictionaries
merged_data = data.copy()  # Start with a copy of the first dictionary
merged_data.update(data_off_diag)  # Update with the second dictionary

Keys in data: dict_keys(['EEEEEE', 'LLLLLL', 'EELLEE', 'ELEELE', 'ELLLLE', 'LELLEL', 'LLEELL', 'LEEEEL'])
Keys in data_off_diag: dict_keys(['EEELLL', 'LLLEEE', 'ELELEL', 'LELELE', 'EELELL', 'LLELEE', 'ELLEEL', 'LEELLE'])


In [10]:
# Stabilizer calculations
def compute_stabilizers(pauli_expectations):
    """
    Computes the stabilizers and bounds the fidelity.
    
    Parameters:
    pauli_expectations: dict containing expectation values for 'S1', 'S_k'

    Returns:
    Fidelity bound based on the stabilizer formalism.
    """

    # Extract the Pauli expectation values (S1 and Sk)
    S1_expectation = pauli_expectations['S1']  # Expectation of S1 = <sigma_x1 sigma_x2 ... sigma_xN>

    # Expectation of Sk = <sigma_z_(k-1) sigma_z_k> for k >= 2
    Sk_expectations = pauli_expectations['Sk']  # Should be a list of S_k expectation values for k >= 2

    # Fidelity bound calculation
    fidelity_bound = 0.5 * (S1_expectation + np.prod([1 + Sk for Sk in Sk_expectations]) - 1)

    return fidelity_bound


# Function to compute Pauli expectation values from the data
def calculate_pauli_expectations(data, ignore_rel_phase=False):
    """
    Calculate the expectation values for the Pauli operators from the given data.

    Parameters:
    data: dict containing the density matrix elements in the E and L basis.

    Returns:
    A dictionary with computed expectation values for S1 and Sk.
    """

    # Extract relevant components
    EEEEEE = data['EEEEEE']['real_integral_value']
    LLLLLL = data['LLLLLL']['real_integral_value']
    EELLEE = data['EELLEE']['real_integral_value']
    ELEELE = data['ELEELE']['real_integral_value']
    ELLLLE = data['ELLLLE']['real_integral_value']
    LELLEL = data['LELLEL']['real_integral_value']
    LLEELL = data['LLEELL']['real_integral_value']
    LEEEEL = data['LEEEEL']['real_integral_value']
    EEELLL = data['EEELLL'][
        'real_integral_value'] + data['EEELLL']['imag_integral_value'] * 1j
    LLLEEE = data['LLLEEE'][
        'real_integral_value'] - data['LLLEEE']['imag_integral_value'] * 1j
    ELELEL = data['ELELEL'][
        'real_integral_value'] + data['ELELEL']['imag_integral_value'] * 1j
    LELELE = data['LELELE'][
        'real_integral_value'] - data['LELELE']['imag_integral_value'] * 1j
    EELELL = data['EELELL'][
        'real_integral_value'] + data['EELELL']['imag_integral_value'] * 1j
    LLELEE = data['LLELEE'][
        'real_integral_value'] - data['LLELEE']['imag_integral_value'] * 1j
    ELLEEL = data['ELLEEL'][
        'real_integral_value'] + data['ELLEEL']['imag_integral_value'] * 1j
    LEELLE = data['LEELLE'][
        'real_integral_value'] - data['LEELLE']['imag_integral_value'] * 1j

    if ignore_rel_phase:
        if np.abs(LLLEEE) > min(np.abs(EEEEEE), np.abs(LLLLLL)):
            LLLEEE = min(np.abs(EEEEEE), np.abs(LLLLLL))
            EEELLL = np.conj(LLLEEE)
        else:
            LLLEEE = np.abs(LLLEEE)
            EEELLL = np.abs(EEELLL)

    # Calculate the sums for normalization
    total_sum = EEEEEE + LLLLLL + EELLEE + ELEELE + ELLLLE + LELLEL + LLEELL + LEEEEL

    # Compute expectation values for sigma_x and sigma_z
    sigma_x_expectation = (EEELLL + LLLEEE + ELELEL + LELELE + EELELL +
                           LLELEE + ELLEEL +
                           LEELLE) / total_sum if total_sum != 0 else 0
    sigma_z_expectation_1 = (EEEEEE + LLLLLL - EELLEE - ELEELE + ELLLLE -
                             LELLEL - LLEELL +
                             LEEEEL) / total_sum if total_sum != 0 else 0
    sigma_z_expectation_2 = (EEEEEE - LLLLLL + EELLEE - ELEELE - ELLLLE +
                             LELLEL - LLEELL +
                             LEEEEL) / total_sum if total_sum != 0 else 0
    sigma_z_expectation_3 = (EEEEEE - LLLLLL - EELLEE + ELEELE - ELLLLE -
                             LELLEL + LLEELL +
                             LEEEEL) / total_sum if total_sum != 0 else 0

    # Prepare the expectations dictionary
    pauli_expectations = {
        'S1': sigma_x_expectation,
        'Sk':
        [sigma_z_expectation_1, sigma_z_expectation_2,
         sigma_z_expectation_3]  # For k >= 2, we can add more Sk if needed.
    }

    return pauli_expectations


def calculate_pauli_expectation_uncertainties(data, fixed_element_uncertainty, ignore_rel_phase=False):
    """
    Calculate the uncertainties of the Pauli expectation values given a fixed uncertainty per element.

    Parameters:
    data: dict containing the density matrix elements in the E and L basis.
    fixed_element_uncertainty: numerical uncertainty for each real/imag integral value (e.g., 1/16).
    ignore_rel_phase: bool, whether to ignore relative phase for off-diagonal terms.

    Returns:
    A dictionary with uncertainties for S1 and Sk.
    """

    # First, extract all relevant terms
    EEEEEE = data['EEEEEE']['real_integral_value']
    LLLLLL = data['LLLLLL']['real_integral_value']
    EELLEE = data['EELLEE']['real_integral_value']
    ELEELE = data['ELEELE']['real_integral_value']
    ELLLLE = data['ELLLLE']['real_integral_value']
    LELLEL = data['LELLEL']['real_integral_value']
    LLEELL = data['LLEELL']['real_integral_value']
    LEEEEL = data['LEEEEL']['real_integral_value']

    EEELLL = data['EEELLL']['real_integral_value'] + 1j * data['EEELLL']['imag_integral_value']
    LLLEEE = data['LLLEEE']['real_integral_value'] - 1j * data['LLLEEE']['imag_integral_value']
    ELELEL = data['ELELEL']['real_integral_value'] + 1j * data['ELELEL']['imag_integral_value']
    LELELE = data['LELELE']['real_integral_value'] - 1j * data['LELELE']['imag_integral_value']
    EELELL = data['EELELL']['real_integral_value'] + 1j * data['EELELL']['imag_integral_value']
    LLELEE = data['LLELEE']['real_integral_value'] - 1j * data['LLELEE']['imag_integral_value']
    ELLEEL = data['ELLEEL']['real_integral_value'] + 1j * data['ELLEEL']['imag_integral_value']
    LEELLE = data['LEELLE']['real_integral_value'] - 1j * data['LEELLE']['imag_integral_value']

    if ignore_rel_phase:
        if np.abs(LLLEEE) > min(np.abs(EEEEEE), np.abs(LLLLLL)):
            LLLEEE = min(np.abs(EEEEEE), np.abs(LLLLLL))
            EEELLL = np.conj(LLLEEE)
        else:
            LLLEEE = np.abs(LLLEEE)
            EEELLL = np.abs(EEELLL)

    # Sum for normalization
    total_sum = EEEEEE + LLLLLL + EELLEE + ELEELE + ELLLLE + LELLEL + LLEELL + LEEEEL

    # -------------------------------
    # Define list of elements involved
    # -------------------------------
    diag_terms = [EEEEEE, LLLLLL, EELLEE, ELEELE, ELLLLE, LELLEL, LLEELL, LEEEEL]
    offdiag_terms = [EEELLL, LLLEEE, ELELEL, LELELE, EELELL, LLELEE, ELLEEL, LEELLE]

    # Each real or imaginary part comes with fixed_element_uncertainty

    # -------------------------------
    # Calculate uncertainty on total_sum
    # -------------------------------
    delta_total_sum = np.sqrt(len(diag_terms)) * fixed_element_uncertainty

    # -------------------------------
    # Calculate uncertainty on numerator of sigma_x (S1)
    # -------------------------------
    delta_numerator_sx = np.sqrt(2 * len(offdiag_terms)) * fixed_element_uncertainty  # 2x because real and imag parts separately contribute

    # Propagate error:
    if np.abs(total_sum) > 1e-12:
        relative_error_total_sum = delta_total_sum / np.abs(total_sum)
        relative_error_numerator_sx = delta_numerator_sx / np.abs(
            sum(offdiag_terms)
        ) if np.abs(sum(offdiag_terms)) > 1e-10 else 0

        # Error propagation for division:
        delta_sigma_x = np.abs((sum(offdiag_terms)) / total_sum) * np.sqrt(
            relative_error_numerator_sx ** 2 + relative_error_total_sum ** 2
        )
    else:
        delta_sigma_x = 0.0

    # -------------------------------
    # Calculate uncertainty on sigma_z components (Sk)
    # -------------------------------
    # Numerators for sigma_z expectations:
    numerators_sz = [
        EEEEEE + LLLLLL - EELLEE - ELEELE + ELLLLE - LELLEL - LLEELL + LEEEEL,
        EEEEEE - LLLLLL + EELLEE - ELEELE - ELLLLE + LELLEL - LLEELL + LEEEEL,
        EEEEEE - LLLLLL - EELLEE + ELEELE - ELLLLE - LELLEL + LLEELL + LEEEEL
    ]

    delta_sigma_z_list = []
    for numerator_sz in numerators_sz:
        if np.abs(total_sum) > 1e-12:
            relative_error_numerator_sz = np.sqrt(len(diag_terms)) * fixed_element_uncertainty / np.abs(numerator_sz) if np.abs(numerator_sz) > 1e-12 else 0
            delta_sigma_z = np.abs(numerator_sz / total_sum) * np.sqrt(
                relative_error_numerator_sz ** 2 + relative_error_total_sum ** 2
            )
        else:
            delta_sigma_z = 0.0
        delta_sigma_z_list.append(delta_sigma_z)

    # -------------------------------
    # Return the uncertainties
    # -------------------------------
    pauli_expectation_uncertainties = {
        'S1': delta_sigma_x,
        'Sk': delta_sigma_z_list
    }

    return pauli_expectation_uncertainties

def compute_fidelity_uncertainty(pauli_expectations, pauli_uncertainties):
    """
    Compute the uncertainty in the fidelity bound based on uncertainties in the Pauli expectations.

    Parameters:
    pauli_expectations: dict with 'S1' and 'Sk'
    pauli_uncertainties: dict with uncertainties for 'S1' and 'Sk'

    Returns:
    Uncertainty on the fidelity bound.
    """

    S1 = pauli_expectations['S1']
    Sk_list = pauli_expectations['Sk']

    S1_unc = pauli_uncertainties['S1']
    Sk_unc_list = pauli_uncertainties['Sk']

    # Product term: prod(1 + Sk)
    prod_term = np.prod([1 + Sk for Sk in Sk_list])

    # Error propagation for product:
    if prod_term != 0:
        relative_variance_prod = sum(
            (Sk_unc / (1 + Sk))**2 for Sk, Sk_unc in zip(Sk_list, Sk_unc_list)
        )
    else:
        relative_variance_prod = 0

    # Now propagate error in full expression
    # F = 0.5 * (S1 + prod_term - 1)
    fidelity_variance = 0.25 * (
        S1_unc**2 + (prod_term**2) * relative_variance_prod
    )

    fidelity_uncertainty = np.sqrt(fidelity_variance)

    return fidelity_uncertainty


# Function to load pkl files and extract fidelity and detuning
def load_fidelity_and_detuning(folder_path, n_start):
    """
    Loads pkl files from a folder, extracts fidelity and detuning.

    Parameters:
    folder_path: str, path to the folder containing pkl files
    n_start: int, the starting number to filter relevant files

    Returns:
    A list of (detuning, fidelity) pairs.
    """
    fidelity_detuning_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".pkl"):
            file_path = os.path.join(folder_path, filename)
            print(f"Processing file: {filename}")

            try:
                # Extract detuning from the filename based on n_start value
                if f'nstart{n_start}' in filename:
                    if n_start == 2:
                        detuning_str = filename.split('twophotdet0_')[1].split('.pkl')[0].replace('p', '.')
                    elif n_start == 1:
                        detuning_str = filename.split('twophotdet')[1].split('_')[0].replace('p', '.')
                    detuning = float(detuning_str)

                    # Load the .pkl file
                    with open(file_path, 'rb') as f:
                        data = pickle.load(f)

                    # Calculate the Pauli expectations using the structured data
                    pauli_expectations = calculate_pauli_expectations(data)

                    # Compute the fidelity bound using stabilizer formalism
                    fidelity = compute_stabilizers(pauli_expectations)

                    # Store the detuning and fidelity pair
                    fidelity_detuning_data.append((detuning, fidelity))
                else:
                    print(f"File does not match n_start={n_start}: {filename}")
            except (ValueError, IndexError) as e:
                print(f"Error parsing detuning from {file_path}: {e}")

    return fidelity_detuning_data

'''
# Example usage
folder_path = "photon_correlations/n_2/det_sweep/"  # Replace with your folder path
n_start = 1  # Set your desired n_start value (1 or 2)
fidelity_detuning_pairs = load_fidelity_and_detuning(folder_path, n_start)

# Sorting the data by detuning (ascending)
fidelity_detuning_pairs.sort(key=lambda x: x[0])  # Sort by detuning

# Display sorted results
for detuning, fidelity in fidelity_detuning_pairs:
    print(f"Detuning: {detuning}, Fidelity: {fidelity}")
'''

'\n# Example usage\nfolder_path = "photon_correlations/n_2/det_sweep/"  # Replace with your folder path\nn_start = 1  # Set your desired n_start value (1 or 2)\nfidelity_detuning_pairs = load_fidelity_and_detuning(folder_path, n_start)\n\n# Sorting the data by detuning (ascending)\nfidelity_detuning_pairs.sort(key=lambda x: x[0])  # Sort by detuning\n\n# Display sorted results\nfor detuning, fidelity in fidelity_detuning_pairs:\n    print(f"Detuning: {detuning}, Fidelity: {fidelity}")\n'

In [4]:
def calc_fidelity_rate(data):
    # Load data from the provided file path
    try:
        # Calculate rate based on specific fields
        rate = (data["EEEEEE"]["real_integral_value"] + data["LLLLLL"]["real_integral_value"]) * 2 * 2.1 * np.pi        
        # Calculate Pauli expectations and fidelity lower bound (fid_lb)
        paulis = calculate_pauli_expectations(data, True)
        fid_lb = compute_stabilizers(paulis)
    except KeyError as e:
        print(f"Key error: {e} not found in data.")
        return None, None, None
    
    return fid_lb,rate

In [6]:
# Loading and computing fidelity bounds as per the setup
new_data_bm = load_pkl(
    "photon_correlations/n_3/n3_results_diag_nstart2_bm_vstlength1_b0p07_twophotdet0_0.pkl"
)
new_data_no_spontaneous = load_pkl("photon_correlations/n_3/n3_results_diag_nstart2_nospont_vstlength1_b0p07_twophotdet0_0.pkl")
print(new_data_no_spontaneous)
new_data_no_spont_no_x = load_pkl("photon_correlations/n_3/n3_results_nstart2_nospont_noxstate_svstlength1_b0p07_twophotdet0_0.pkl")
data_ft_bm = load_pkl("photon_correlations/n_3/n3_results_ftbmpulse_nstart2_spontTrue_vstlength1_b0_twophotdet0_0.pkl")
data_ft_bm_no_spont = load_pkl("photon_correlations/n_3/n3_results_ftbmpulse_nstart2_spontFalse_vstlength1_b0_twophotdet0_0.pkl")
data_ft_bm_bp7 = load_pkl("photon_correlations/n_3/n3_results_ftbmpulse_nstart2_spontTrue_vstlength1_b0p07_twophotdet0_0.pkl")


#Calculate and print fidelity bounds for each data set
for label, data in [("Standard", merged_data), ("No Spontaneous Emission", new_data_no_spontaneous),
                    ("No Spont. & No X States", new_data_no_spont_no_x), ("Flat-top BM B=0", data_ft_bm), ("Flat-top BM B=0 No Spont.", data_ft_bm_no_spont), ("Flat-top BM B=0.07", data_ft_bm_bp7)]:
    fid,rate=calc_fidelity_rate(data)
    print(f"{label} Fidelity bound : {fid}, Rate: {rate}")

{'EEEEEE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det_2': 0, 'two_photon_det_4': 0, 'real_integral_value': 1.96596451200752e-05, 'imag_integral_value': 0.0, 'n_time_steps': 16}, 'LLLLLL': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det_2': 0, 'two_photon_det_4': 0, 'real_integral_value': 2.0355271870518158e-05, 'imag_integral_value': 0.0, 'n_time_steps': 16}, 'EELLEE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det_2': 0, 'two_photon_det_4': 0, 'real_integral_value': 4.3229420780278874e-07, 'imag_integral_value': 0.0, 'n_time_steps': 16}, 'ELEELE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det_2': 0, 'two_photon_det_4': 0, 'real_integral_value': 2.071961695195802e-08, 'imag_integral_value': 0.0, 'n_time_steps': 16}, 'ELLLLE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det_2': 0, 'two_photon_det_4': 0, 'real_integral_value': 3.143262922666142e-0

In [5]:
new_data_bm_ramp = load_pkl(
    "photon_correlations/numerical_precision/n_3/n3_results_nstart1_spontTrue_vstlength1_b0p07_omega_sti30_30_68384c6827cd9ac5.pkl"
)
print(new_data_bm_ramp)
fid, rate = calc_fidelity_rate(new_data_bm_ramp)
print(f"BM Ramp Fidelity bound : {fid}, Rate: {rate}")

{'EEEEEE': {'n_start': 1, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [188.49555921538757, 188.49555921538757], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.7, 'vst_fall_time': 0.2, 'real_integral_value': 1.4753454409009651e-05, 'imag_integral_value': 0.0, 'n_time_steps': 24}, 'LLLLLL': {'n_start': 1, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [188.49555921538757, 188.49555921538757], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.7, 'vst_fall_time': 0.2, 'real_integral_value': 1.8717337749604167e-05, 'imag_integral_value': 0.0, 'n_time_steps': 24}, 'EELLEE': {'n_start': 1, 'bfield_sp

In [11]:
new_data_bm_ramp = load_pkl(
    "/Users/ernst/Desktop/rb_photon_prod_dev/run_correlation_calc/saved_data_timebin/photon_correlations/numerical_precision/n_3/sinsquared/n3_results_nstart1_spontTrue_vstlength1_b0p07_omega_sti50_50_ac73f7129f941c81.pkl"
)
print(new_data_bm_ramp)
fid, rate = calc_fidelity_rate(new_data_bm_ramp)
print(f"BM Ramp Fidelity bound : {fid}, Rate: {rate}")

{'EEEEEE': {'n_start': 1, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'sinsquared', 'vst_rise_time': 0.7, 'vst_fall_time': 0.2, 'real_integral_value': 1.5093441847096736e-05, 'imag_integral_value': 0.0, 'n_time_steps': 24}, 'LLLLLL': {'n_start': 1, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'sinsquared', 'vst_rise_time': 0.7, 'vst_fall_time': 0.2, 'real_integral_value': 1.4614996111350618e-05, 'imag_integral_value': 0.0, 'n_time_steps': 24}, 'EELLEE': {'n_start': 1, 'bfield_split': '0p07', 'l

In [12]:
uncertainty_stabiliser = calculate_pauli_expectation_uncertainties(
    new_data_bm_ramp, 1e-7, True)
print(uncertainty_stabiliser)
stabilisers = calculate_pauli_expectations(new_data_bm_ramp, True)
fid_uncertainty = compute_fidelity_uncertainty(stabilisers,
                                               uncertainty_stabiliser)

print(fid_uncertainty)

{'S1': 0.014653625483455645, 'Sk': [0.012474717889385763, 0.009028069576370293, 0.00902761595172452]}
0.015810514883468373


In [7]:
new_n3_data={'EEEEEE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 1.928542310545819e-05, 'imag_integral_value': 0.0, 'n_time_steps': 12}, 'LLLLLL': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 1.7191780946965148e-05, 'imag_integral_value': 0.0, 'n_time_steps': 12}, 'EELLEE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 5.564938656772494e-07, 'imag_integral_value': 0.0, 'n_time_steps': 12}, 'ELEELE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 5.818708931384077e-08, 'imag_integral_value': 0.0, 'n_time_steps': 12}, 'ELLLLE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 3.3866110040368146e-07, 'imag_integral_value': 0.0, 'n_time_steps': 12}, 'LELLEL': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 7.338063072948256e-08, 'imag_integral_value': 0.0, 'n_time_steps': 12}, 'LLEELL': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 1.8290949475727917e-07, 'imag_integral_value': 0.0, 'n_time_steps': 12}, 'LEEEEL': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 1.6998202083605946e-07, 'imag_integral_value': 0.0, 'n_time_steps': 12}, 'EEELLL': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 5.381554067654246e-06, 'imag_integral_value': 1.6003413012338457e-05, 'n_time_steps': 12}, 'LLLEEE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 5.381554067654246e-06, 'imag_integral_value': 1.6003413012338457e-05, 'n_time_steps': 12}, 'ELELEL': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': -3.6037697484433083e-09, 'imag_integral_value': -2.0997236460892416e-09, 'n_time_steps': 12}, 'LELELE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': -3.6037697484433083e-09, 'imag_integral_value': -2.0997236460892416e-09, 'n_time_steps': 12}, 'EELELL': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': -3.6037697484433083e-09, 'imag_integral_value': -2.0997236460892416e-09, 'n_time_steps': 12}, 'LLELEE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': -3.6037697484433083e-09, 'imag_integral_value': -2.0997236460892416e-09, 'n_time_steps': 12}, 'ELLEEL': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 4.474237312955295e-09, 'imag_integral_value': 4.759764096525902e-09, 'n_time_steps': 12}, 'LEELLE': {'n_start': 2, 'bfield_split': '0p07', 'length_stirap': 1, 'two_photon_det': [0, 0, 0, 0], 'omega_stirap': [314.1592653589793, 314.1592653589793], 'omega_rot_stirap': [573.4, 754.1772283803055, 1122.6363636363635, 991.7272727272727], 'spontaneous_emission': True, 'chirped_pulse': True, 'vstirap_pulse_shape': 'flattop_blackman', 'vst_rise_time': 0.15, 'vst_fall_time': 0.1, 'real_integral_value': 4.474237312955295e-09, 'imag_integral_value': 4.759764096525902e-09, 'n_time_steps': 12}}